In [1]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.docstore.document import Document
from nltk.corpus import wordnet
import torch

# Ensure NLTK resources are downloaded
import nltk
nltk.download('wordnet')

# Authenticate with the Kaggle API
api = KaggleApi()
api.authenticate()


/home/nealsharma/.local/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nealsharma/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Download the "All The News" dataset from Kaggle
api.dataset_download_files('asad1m9a9h6mood/news-articles', path='data/', unzip=True)

# Load and process the dataset
file_path = 'data/Articles.csv'  # Adjust the path based on dataset name
df = pd.read_csv(file_path, encoding='ISO-8859-1')


Dataset URL: https://www.kaggle.com/datasets/asad1m9a9h6mood/news-articles


In [3]:
# Convert documents into the required format for Chroma 
document_list = [
    Document(
        page_content=row['Article'],
        metadata={
            'date': row['Date'],
            'heading': row['Heading'],
            'news_type': row['NewsType']
        }
    )
    for _, row in df.iterrows()
]

# TODO Truncate document list because it takes too long to process
if len(document_list) > 1000:
    document_list = document_list[:1000]


In [4]:
# Embed documents using SentenceTransformer
embedder = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

# Create a vector store using Chroma
vector_store = Chroma.from_documents(document_list, embeddings)


/home/nealsharma/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_5344/1819464542.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')


In [5]:
# Start of 2.2
# Function to expand the query using WordNet

def expand_query_with_synonyms(query):
    synonyms = set()
    for word in query.split():
        for syn in wordnet.synsets(word):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
    expanded_query = query + " " + " ".join(synonyms)
    return expanded_query


In [6]:
# Expand the initial query
query = "Tell me about the latest news in the world"
expanded_query = expand_query_with_synonyms(query)

# Perform semantic search and retrieve relevant documents
retrieved_docs = vector_store.similarity_search(expanded_query, k=5)

# Combine top-ranked documents into a context string
context = " ".join([doc.page_content for doc in retrieved_docs[:2]])  # Limiting to top 2 documents


In [7]:
# # Get the Hugging Face token from the environment variables
# hf_token = os.getenv("HUGGING_FACE_TOKEN")
# #print(hf_token)

# # Initialize the LLM (using Mistral or GPT-2 as an example)
# #llm = pipeline("text-generation", model="gpt2")  # Replace with another model if needed
# #for VM offline model usage
# #llm = pipeline("text-generation", model="/home/nealsharma/.cache/huggingface/hub/models--mistralai--Mistral-7B-v0.1/snapshots/7231864981174d9bee8c7687c24c8344414eae6b", batch_size = 50)

# llm = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1", token=hf_token, batch_size = 50)


# # Generate the final response
# response = llm(context, max_new_tokens=100, do_sample=True, top_k=50)[0]['generated_text']
# print(response)

In [8]:
# Set the correct model path
model_path = "/home/nealsharma/llm/Mistral-7B-v0.1"

In [ ]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Move the model to the appropriate device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # CHANGED

# Tokenize the context
inputs = tokenizer(context, return_tensors="pt").to(device)


In [ ]:
# Generate text using the loaded model
with torch.no_grad():  # Disables gradient computation for faster inference
    outputs = model.generate(
        inputs['input_ids'],
        max_new_tokens=100,  # Generate up to 100 new tokens
        do_sample=True,      # Enable sampling for more varied results
        top_k=50,            # Limit the sampling to the top 50 tokens
        temperature=0.7      # Control the randomness (higher = more random)
    )

# Decode the generated tokens back into text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)

